<a href="https://colab.research.google.com/github/custom-hyper/DeFi_Lama_API/blob/master/DeFi_Lama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Setup

In [9]:
#@title  Install  Packages { display-mode: "code" }
!pip install requests
!pip install json
!pip install psycopg2
!pip install pandas


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [2]:
#@title Connect to Drive { display-mode: "code" }
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#@title  Import Libraries { display-mode: "code" }
import requests
import json
from google.colab import data_table
import sqlite3 as sq
import pandas as pd
import logging
import pandas as pd
import logging
import time
from tqdm.notebook import tqdm
from datetime import datetime
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [4]:
#@title  Connect { display-mode: "code" }
path = '/content/drive/MyDrive/workspace/pipelines/db/' #@param {type: 'string'
database = "db.sqlite" #@param {type: 'string'
conn_SQLite = sq.connect(path+database)

#@title   Show Available Tables { display-mode: "code" }
cursor = conn_SQLite.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.execute("DROP TABLE deFi_lama_general;")
cursor.fetchall()

[]

# Data Extraction

Please use  df['slug'] as primary key if loaded into SQL light

## Protocols ID List

In [16]:
#@title Call API { display-mode: "code" }



#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
DeFi = json.loads(output)
full_talbe = pd.json_normalize(DeFi,max_level=4)
full_talbe['id'] = full_talbe['symbol']  + '.' + full_talbe['name']
full_talbe['id'].replace(' ','')
full_talbe['Notes'] = ''

#UNIX timestamp: created
full_talbe['UNIX'] = int( time.time())



full_talbe.head(1)

,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,chainTvls.zkSync,chainTvls.zkSync-offers,chainTvls.Metis-pool2,chainTvls.Harmony-Treasury,chainTvls.Metis-Treasury,chainTvls.Velas-staking,chainTvls.Celo-Treasury,chainTvls.Ergo-staking,Notes,UNIX
0,CRV.Curve,Curve,0xD533a949740bb3306d119CC777fa900bA034cd52,CRV,https://curve.fi,Curve is a decentralized exchange liquidity po...,Multi-Chain,https://icons.llama.fi/curve.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650754198


In [17]:
#@title Save To Directory{ display-mode: "code" }
table = "deFi_lama_general"
sfile = "deFi_lama_general.csv"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
full_talbe.to_csv(path + sfile)

In [18]:
#@title Load to SQLite { display-mode: "code" }


full_talbe.to_sql(table, conn_SQLite , if_exists='append', index=False) 

df = pd.read_sql('select * from {}'.format(sfile), conn_SQLite)


InterfaceError: ignored

In [29]:
full_talbe


,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,chainTvls.zkSync,chainTvls.zkSync-offers,chainTvls.Metis-pool2,chainTvls.Harmony-Treasury,chainTvls.Metis-Treasury,chainTvls.Velas-staking,chainTvls.Celo-Treasury,chainTvls.Ergo-staking,Notes,UNIX
0,CRV.Curve,Curve,0xD533a949740bb3306d119CC777fa900bA034cd52,CRV,https://curve.fi,Curve is a decentralized exchange liquidity po...,Multi-Chain,https://icons.llama.fi/curve.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
1,LDO.Lido,Lido,0x5a98fcbea516cf06857215779fd812ca3bef1b32,LDO,https://lido.fi/,"Liquidity for staked assets. Daily rewards, no...",Multi-Chain,https://icons.llama.fi/lido.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
2,ANC.Anchor,Anchor,-,ANC,https://anchorprotocol.com/,Money market on Terra,Multi-Chain,https://icons.llama.fi/anchor.jpg,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
3,MKR.MakerDAO,MakerDAO,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,MKR,https://makerdao.com/,"Builders of Dai, a digital currency that can b...",Ethereum,https://icons.llama.fi/makerdao.jpg,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
4,CVX.Convex Finance,Convex Finance,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,CVX,https://www.convexfinance.com/,Convex simplifies your Curve-boosting experien...,Ethereum,https://icons.llama.fi/convex-finance.jpg,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1510,PUSH.EPNS,EPNS,0xf418588522d5dd018b425E472991E52EBBeEEEEE,PUSH,https://epns.io,The native communication layer of Web3. Send /...,Ethereum,https://icons.llama.fi/epns.jpg,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
1511,TNODE.Trusted Node,Trusted Node,bsc:0x7f12A37B6921FFac11FAB16338B3ae67EE0c462b,TNODE,https://trustednode.io,Tnode token ables you to: Participate in the g...,Multi-Chain,https://icons.llama.fi/trusted-node.png,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
1512,LION.Scrub Finance,Scrub Finance,cronos:0x49fB98F9b4a3183Cd88e7a115144fdf00fa6fB95,LION,https://scrub.finance,Scrub Finance is the first deflationary algori...,Cronos,https://icons.llama.fi/scrub-finance.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
1513,SCNR.Swapscanner,Swapscanner,klaytn:0x8888888888885b073f3c81258c27e83db228d5f3,SCNR,https://swapscanner.io,Swapscanner is a next-generation DEX Aggregato...,Klaytn,https://icons.llama.fi/swapscanner.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207


In [19]:
#@title Show Stats { display-mode: "code" }
full_talbe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1515 entries, 0 to 1514
Columns: 305 entries, id to UNIX
dtypes: float64(279), int64(1), object(25)
memory usage: 3.5+ MB


 ## TVL Timeseries

In [20]:
#@title Call API  { display-mode: "code" }

#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
table = json.loads(output)
df = pd.json_normalize(table)
slug = df['slug']

path = "https://api.llama.fi/protocol/"
frames = []


try:
  for i in tqdm(slug):
    response = requests.get(path + i)
    output = response.text
    DeFi = json.loads(output)

    full_table = pd.json_normalize(DeFi['tvl']) 
    full_table['slug'] = i
    

    #results = [full_table]
    frames.append(full_table)

    #Slow down requests
    
    time.sleep(60/50)
    #Status report
    
    
except ConnectionError:
    time.sleep(60)


single = pd.concat(frames) #Careful, this line must be outside of the for loop
#single['date_Standard']  = datetime.fromtimestamp(time).strftime('%Y-%m-%d')


single.head()

  


  0%|          | 0/1515 [00:00<?, ?it/s]

,date,totalLiquidityUSD,slug
0,1581206400,1.163733e+06,curve
1,1581292800,1.372110e+06,curve
2,1581379200,9.607534e+05,curve
3,1581465600,1.006408e+06,curve
4,1581552000,1.258776e+06,curve


In [21]:
#@title Save To Directory{ display-mode: "code" }
sfile = "deFi_lama_tvl.csv"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
single.to_csv(path + sfile)

In [22]:
#@title Load to SQLite { display-mode: "code" }

table = "deFi_lama_tvl"

single.to_sql(table, conn_SQLite , if_exists='replace', index=False) # writes to file

 # Test Connection
df = pd.read_sql('select * from {}'.format(table), conn_SQLite)

df.head()


,date,totalLiquidityUSD,slug
0,1581206400,1.163733e+06,curve
1,1581292800,1.372110e+06,curve
2,1581379200,9.607534e+05,curve
3,1581465600,1.006408e+06,curve
4,1581552000,1.258776e+06,curve


In [23]:
#@title Show Stats { display-mode: "code" }
single.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290154 entries, 0 to 3
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   date               290154 non-null  int64  
 1   totalLiquidityUSD  290154 non-null  float64
 2   slug               290154 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 8.9+ MB


## Protocol General Information Data

In [24]:
#@title Call API  { display-mode: "code" }


#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
DeFi = json.loads(output)
full_talbe = pd.json_normalize(DeFi)
full_talbe['id'] = full_talbe['symbol']  + '.' + full_talbe['name']
full_talbe['id'].replace(' ','')
full_talbe['Notes'] = ''

#UNIX timestamp: created
full_talbe['UNIX'] = int( time.time())



full_talbe.head(1)



,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,chainTvls.zkSync,chainTvls.zkSync-offers,chainTvls.Metis-pool2,chainTvls.Harmony-Treasury,chainTvls.Metis-Treasury,chainTvls.Velas-staking,chainTvls.Celo-Treasury,chainTvls.Ergo-staking,Notes,UNIX
0,CRV.Curve,Curve,0xD533a949740bb3306d119CC777fa900bA034cd52,CRV,https://curve.fi,Curve is a decentralized exchange liquidity po...,Multi-Chain,https://icons.llama.fi/curve.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207


In [25]:
 #@title Save To Directory{ display-mode: "code" }
 
table = "deFi_lama_general"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
full_talbe.to_csv(path+ table)

In [27]:
#@title Dataset Stats { display-mode: "code" }
full_talbe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1515 entries, 0 to 1514
Columns: 305 entries, id to UNIX
dtypes: float64(279), int64(1), object(25)
memory usage: 3.5+ MB


In [28]:
#@title Load to SQLite { display-mode: "code" }

full_talbe.to_sql(table, conn , if_exists='replace', index=False) # writes to file

 # Test Connection
df = pd.read_sql('select * from {}'.format(table), conn)

df.head()

NameError: ignored

In [ ]:
full_talbe.describe()

In [ ]:
#@title Dataset Stats { display-mode: "code" }
full_talbe.info(verbose=True)

 ## Total Combined TVL

In [ ]:
  #@title API call { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table

import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt

path = "https://api.llama.fi/charts"
#slug = 'aave' 
#General Information table
response = requests.get(path)
output = response.text
DeFi = json.loads(output)
totalLiq_talbe = pd.json_normalize(DeFi)
totalLiq_talbe.head()



In [ ]:
 #@title Save To DeFi Lama Dataset Directory{ display-mode: "code" }
 
table = "deFi_lama_total_TVL"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
totalLiq_talbe.to_csv(path+ table)

In [ ]:
#@title Load General Information Data to SQLite { display-mode: "code" }


totalLiq_talbe.to_sql(table, conn , if_exists='replace', index=False) # writes to file

 # Test Connection
df = pd.read_sql('select * from {}'.format(table), conn)

df.head()

In [ ]:
conn.close()

# Feature Engineering